# Excel Format Conversion Testing

This notebook tests the document converter functionality with the Excel sample file to debug markdown rendering issues.

In [ ]:
# Setup and Installation
ROOTDIR = "../.."

# Let's make sure that modules are autoreloaded
%load_ext autoreload
%autoreload 2

# First uninstall existing package (to ensure we get the latest version)
%pip uninstall -y idp_common

# Install the IDP common package with all components in development mode
%pip install -q -e "{ROOTDIR}/lib/idp_common_pkg[dev, all]"

# Check installed version
%pip show idp_common | grep -E "Version|Location"

print("✓ IDP Common package installed with all dependencies")

In [ ]:
# Import required libraries
import os
import sys
import tempfile
import shutil
from pathlib import Path
from idp_common.ocr.document_converter import DocumentConverter
import pandas as pd
from PIL import Image
import io
from IPython.display import display, Markdown, HTML
import matplotlib.pyplot as plt
import numpy as np

print("Successfully imported DocumentConverter and dependencies")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# Create temporary directory for output files
temp_dir = tempfile.mkdtemp(prefix='excel_test_')
print(f"✓ Created temporary directory: {temp_dir}")
print(f"All output files will be saved to this directory")

## 1. Load Excel Sample File

In [ ]:
# Load the Excel sample file
excel_file_path = '../../samples/ProbeScanExport.xlsx'

# Check if file exists
if os.path.exists(excel_file_path):
    print(f"✓ Found Excel file: {excel_file_path}")
    
    # Get file size
    file_size = os.path.getsize(excel_file_path)
    print(f"File size: {file_size:,} bytes ({file_size/1024:.1f} KB)")
    
    # Read file as bytes
    with open(excel_file_path, 'rb') as f:
        excel_bytes = f.read()
    
    print(f"Successfully loaded {len(excel_bytes):,} bytes")
else:
    print(f"❌ Excel file not found: {excel_file_path}")

## 2. Preview Excel Content with Pandas

In [ ]:
# First, let's see what pandas sees in this Excel file
try:
    # Read Excel file with pandas to understand structure
    excel_file = pd.ExcelFile(excel_file_path)
    
    print(f"Sheet names: {excel_file.sheet_names}")
    print(f"Number of sheets: {len(excel_file.sheet_names)}")
    
    # Read each sheet and show basic info
    for sheet_name in excel_file.sheet_names:
        df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
        print(f"\n--- Sheet: {sheet_name} ---")
        print(f"Shape: {df.shape} (rows x columns)")
        print(f"Columns: {list(df.columns)}")
        
        # Show first few rows
        print("\nFirst 3 rows:")
        display(df.head(3))
        
        # Show data types
        print("\nData types:")
        print(df.dtypes)
        
except Exception as e:
    print(f"Error reading Excel file with pandas: {e}")

## 3. Convert Excel to Pages

In [ ]:
# Initialize the DocumentConverter
converter = DocumentConverter(dpi=150)

print(f"DocumentConverter initialized with DPI: {converter.dpi}")
print(f"Page dimensions: {converter.page_width} x {converter.page_height} pixels")
print(f"Margin: {converter.margin} pixels")

In [ ]:
# Convert Excel to pages
try:
    print("Converting Excel file to pages...")
    pages = converter.convert_excel_to_pages(excel_bytes)
    
    print(f"✓ Conversion successful!")
    print(f"Number of pages generated: {len(pages)}")
    
    # Show info about each page
    for i, (img_bytes, page_text) in enumerate(pages):
        print(f"\nPage {i+1}:")
        print(f"  Image size: {len(img_bytes):,} bytes")
        print(f"  Text length: {len(page_text):,} characters")
        print(f"  Text preview: {page_text[:100]}...")
        
except Exception as e:
    print(f"❌ Error during conversion: {e}")
    import traceback
    traceback.print_exc()

## 4. Display Page Text (Raw Markdown)

In [ ]:
# Display the raw markdown text for each page
if 'pages' in locals() and pages:
    for i, (img_bytes, page_text) in enumerate(pages):
        print(f"\n{'='*60}")
        print(f"PAGE {i+1} - RAW MARKDOWN TEXT")
        print(f"{'='*60}")
        print(page_text)
        print(f"\n{'='*60}")
        print(f"END OF PAGE {i+1}")
        print(f"{'='*60}")
        
        # Save each page text to a file for inspection
        markdown_file_path = os.path.join(temp_dir, f'page_{i+1}.md')
        try:
            with open(markdown_file_path, 'w', encoding='utf-8') as f:
                f.write(page_text)
            print(f"✓ Saved page {i+1} markdown to: {markdown_file_path}")
        except Exception as e:
            print(f"❌ Error saving page {i+1} markdown: {e}")
else:
    print("❌ No pages to display")

## 5. Display Page Text (Rendered Markdown)

In [ ]:
# Display the rendered markdown for each page
if 'pages' in locals() and pages:
    for i, (img_bytes, page_text) in enumerate(pages):
        print(f"\n=== PAGE {i+1} - RENDERED MARKDOWN ===")
        
        # Debug: Show first few lines of page text to understand the issue
        print(f"\nDEBUG - First 10 lines of page {i+1}:")
        page_lines = page_text.split('\n')
        for j, line in enumerate(page_lines[:10]):
            print(f"  Line {j+1}: '{line}'")
        
        # Check for table syntax
        table_lines = [line for line in page_lines if line.strip().startswith('|')]
        separator_lines = [line for line in page_lines if '---' in line and '|' in line]
        print(f"\nDEBUG - Table analysis for page {i+1}:")
        print(f"  Lines with |: {len(table_lines)}")
        print(f"  Separator lines: {len(separator_lines)}")
        
        if table_lines:
            print(f"  First table line: '{table_lines[0]}'")
        
        print(f"\nRendering markdown for page {i+1}:")
        display(Markdown(page_text))
        print(f"\n=== END OF PAGE {i+1} RENDERED MARKDOWN ===")
else:
    print("❌ No pages to display")

## 6. Display Page Images

In [ ]:
# Display the generated page images
if 'pages' in locals() and pages:
    print(f"Displaying {len(pages)} generated page images:")
    
    for i, (img_bytes, page_text) in enumerate(pages):
        print(f"\n=== PAGE {i+1} IMAGE ===")
        
        # Load and display image
        try:
            img = Image.open(io.BytesIO(img_bytes))
            
            # Save image to temp directory
            img_file_path = os.path.join(temp_dir, f'page_{i+1}.png')
            img.save(img_file_path, 'PNG')
            print(f"✓ Saved image to: {img_file_path}")
            
            # Display image in notebook
            plt.figure(figsize=(12, 16))
            plt.imshow(img)
            plt.axis('off')
            plt.title(f'Page {i+1} - Generated Image')
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"❌ Error displaying page {i+1} image: {e}")
            
else:
    print("❌ No pages to display")

## 7. Debug the Markdown Conversion Process

In [ ]:
# Let's debug what's happening in the markdown conversion process
if 'pages' in locals() and pages:
    print("=== DEBUGGING THE MARKDOWN CONVERSION ISSUE ===")
    print()
    
    # Let's trace through what the converter is doing
    try:
        # Step 1: Get the raw markdown that should be generated
        import tempfile
        
        with tempfile.NamedTemporaryFile(suffix=".xlsx") as tmp_file:
            tmp_file.write(excel_bytes)
            tmp_file.flush()
            
            # Read all sheets and extract formatted data
            excel_file = pd.ExcelFile(tmp_file.name)
            formatted_elements = []
            
            for sheet_name in excel_file.sheet_names:
                df = pd.read_excel(tmp_file.name, sheet_name=sheet_name)
                
                if df.empty:
                    continue
                    
                # Add sheet header element
                formatted_elements.append({
                    "type": "sheet_header",
                    "sheet_name": sheet_name,
                    "space_before": 20,
                    "space_after": 15,
                })
                
                # Convert DataFrame to formatted table data
                table_data = converter._extract_excel_table_data(df)
                
                if table_data:
                    formatted_elements.append({
                        "type": "excel_table",
                        "data": table_data,
                        "sheet_name": sheet_name,
                        "space_before": 10,
                        "space_after": 20,
                    })
            
            # Step 2: Generate the enhanced markdown (this should be proper markdown)
            raw_markdown = converter._generate_enhanced_excel_markdown(formatted_elements)
            
            print("STEP 1: Raw markdown generated by _generate_enhanced_excel_markdown():")
            print("(This should be proper markdown with | tables |)")
            print("="*60)
            print(raw_markdown[:1000])  # First 1000 chars
            print("="*60)
            
            # Step 3: See what _convert_markdown_to_pages does to it
            print("\nSTEP 2: What _convert_markdown_to_pages() does:")
            print("(This is where the problem occurs - it converts markdown to ASCII tables)")
            
            # Let's see what the _parse_markdown_content_with_tables function does
            formatted_lines = converter._parse_markdown_content_with_tables(raw_markdown)
            
            print(f"Number of formatted lines: {len(formatted_lines)}")
            print("\nFirst 10 formatted lines:")
            for i, line in enumerate(formatted_lines[:10]):
                print(f"Line {i}: {line}")
            
            print("\nLooking for table-related lines:")
            table_lines = [line for line in formatted_lines if 'table' in line.get('type', '')]
            print(f"Found {len(table_lines)} table-related lines")
            if table_lines:
                for i, line in enumerate(table_lines[:5]):
                    print(f"Table line {i}: {line}")
            
    except Exception as e:
        print(f"❌ Error in debugging: {e}")
        import traceback
        traceback.print_exc()
else:
    print("❌ No pages to debug")

## 8. Compare with Direct Pandas Output

In [ ]:
# Let's test pandas to_markdown() directly to see what it should produce
try:
    # Read the first sheet
    df = pd.read_excel(excel_file_path, sheet_name=0)
    
    print("=== DIRECT PANDAS to_markdown() OUTPUT ===")
    print("This is what pandas produces directly (for comparison):")
    print()
    
    # Test with the same options used in the converter
    markdown_output = df.head(10).to_markdown(
        index=False, 
        tablefmt="pipe", 
        stralign="left", 
        numalign="right"
    )
    print("Raw pandas output:")
    print(markdown_output)
    
    print("\n" + "="*50)
    print("Rendered pandas output:")
    display(Markdown(markdown_output))
    
    # Save pandas output for comparison
    pandas_file_path = os.path.join(temp_dir, 'pandas_direct_output.md')
    try:
        with open(pandas_file_path, 'w', encoding='utf-8') as f:
            f.write(markdown_output)
        print(f"\n✓ Saved direct pandas output to: {pandas_file_path}")
    except Exception as e:
        print(f"❌ Error saving pandas output: {e}")
    
except Exception as e:
    print(f"❌ Error testing pandas to_markdown(): {e}")

## 8. Analysis and Summary

In [ ]:
# Analyze the results
if 'pages' in locals() and pages:
    print("=== ANALYSIS ===")
    print(f"✓ Excel file loaded: {excel_file_path}")
    print(f"✓ Document converter initialized")
    print(f"✓ Generated {len(pages)} pages")
    print(f"✓ Output directory: {temp_dir}")
    
    # Analyze each page
    for i, (img_bytes, page_text) in enumerate(pages):
        lines = page_text.split('\n')
        table_lines = [line for line in lines if line.strip().startswith('|')]
        
        print(f"\nPage {i+1} Statistics:")
        print(f"  Total lines: {len(lines):,}")
        print(f"  Table lines: {len(table_lines):,}")
        print(f"  Characters: {len(page_text):,}")
        print(f"  Image size: {len(img_bytes):,} bytes")
    
    print(f"\n=== FILES CREATED IN {temp_dir} ===")
    created_files = []
    
    # Check for created files in temp directory
    if os.path.exists(temp_dir):
        for file in os.listdir(temp_dir):
            file_path = os.path.join(temp_dir, file)
            if os.path.isfile(file_path):
                size = os.path.getsize(file_path)
                print(f"  {file}: {size:,} bytes")
                created_files.append(file)
    
    print(f"\n=== DEBUGGING NOTES ===")
    print("1. Compare the 'Raw Markdown Text' with the 'Rendered Markdown' to see formatting issues")
    print("2. Compare the 'Rendered Markdown' with the 'Generated Images' to see rendering problems")
    print("3. Compare with the 'Direct Pandas Output' to see what should be expected")
    print(f"4. Check the files in {temp_dir} for detailed inspection")
    print(f"5. To clean up: rm -rf {temp_dir}")
    
else:
    print("❌ No analysis possible - no pages generated")

In [ ]:
# Optional: Clean up temporary directory
# Uncomment the lines below if you want to automatically clean up
# print(f"Cleaning up temporary directory: {temp_dir}")
# shutil.rmtree(temp_dir)
# print("✓ Temporary directory cleaned up")

print(f"\n📁 Temporary files are available at: {temp_dir}")
print("💡 To manually clean up later, run: rm -rf " + temp_dir)